# A* Search

In [7]:
from collections import namedtuple
from random import choice
from tqdm.auto import tqdm
import numpy as np
import heapq

PUZZLE_DIM = 5
action = namedtuple('Action', ['pos1', 'pos2'])

goal_state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))

In [8]:
# Helper functions
def available_actions(state: np.ndarray) -> list['Action']:
    x, y = [int(_[0]) for _ in np.where(state == 0)]
    actions = list()
    if x > 0:
        actions.append(action((x, y), (x - 1, y)))
    if x < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x + 1, y)))
    if y > 0:
        actions.append(action((x, y), (x, y - 1)))
    if y < PUZZLE_DIM - 1:
        actions.append(action((x, y), (x, y + 1)))
    return actions

def do_action(state: np.ndarray, action: 'Action') -> np.ndarray:
    new_state = state.copy()
    new_state[action.pos1], new_state[action.pos2] = new_state[action.pos2], new_state[action.pos1]
    return new_state

def manhattan_distance(state: np.ndarray) -> int:
    """Calculate the Manhattan distance heuristic for the n-puzzle."""
    total_distance = 0
    for x in range(PUZZLE_DIM):
        for y in range(PUZZLE_DIM):
            value = state[x, y]
            if value != 0:  # Don't compute distance for the empty tile
                target_x, target_y = divmod(value - 1, PUZZLE_DIM)
                total_distance += abs(target_x - x) + abs(target_y - y)
    return total_distance

def a_star(start_state: np.ndarray):
    # Priority queue for the A* frontier
    pq = []
    number_of_actions_considered = 0
    # Initial node with cost 0 and heuristic
    start_h = manhattan_distance(start_state)
    heapq.heappush(pq, (start_h, 0, start_state.tobytes(), []))
    
    # Visited states
    visited = set()
    visited.add(start_state.tobytes())
    
    while pq:
        f, g, current_state_bytes, path = heapq.heappop(pq) 
        current_state = np.frombuffer(current_state_bytes, dtype=int).reshape((PUZZLE_DIM, PUZZLE_DIM))
        
        # Check if we reached the goal
        if np.array_equal(current_state, goal_state):
            return path, g, number_of_actions_considered  # Return the sequence of moves and cost
        
        # Generate successors
        for act in available_actions(current_state):
            new_state = do_action(current_state, act)
            new_state_bytes = new_state.tobytes()
            if new_state_bytes not in visited:
                number_of_actions_considered += 1
                visited.add(new_state_bytes)
                new_g = g + 0.5 # changed from 1 to 0.5
                new_h = manhattan_distance(new_state)
                heapq.heappush(pq, (new_g + new_h, new_g, new_state_bytes, path + [act]))
    
    return None, None  # If no solution is found

In [9]:
# Randomize the initial state
RANDOMIZE_STEPS = 500  
state = np.array([i for i in range(1, PUZZLE_DIM**2)] + [0]).reshape((PUZZLE_DIM, PUZZLE_DIM))
for r in tqdm(range(RANDOMIZE_STEPS), desc='Randomizing'):
    state = do_action(state, choice(available_actions(state)))

solution_path, solution_cost,number_of_actions = a_star(state)
print(f"Cost: {number_of_actions}, Quality: {len(solution_path)}")

Randomizing:   0%|          | 0/500 [00:00<?, ?it/s]

Cost: 1149561, Quality: 84
